In [2]:
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from nltk.tokenize import word_tokenize
from nltk import pos_tag , ne_chunk
from nltk.corpus import stopwords 
import nltk
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score,classification_report
from sklearn.ensemble import RandomForestClassifier

In [26]:
data = pd.read_excel("SBM Raw Dump.xlsx")

In [27]:
data.head(2)

Case ID Issue Date      Time      Contact Method            Full Name  \
0  1284734 2019-02-01  06:07:48       Inbound Email         Mr. Shane NA   
1  1284736 2019-02-01  08:37:24  Inbound Phone Call  Ms. Bobbi Rodriguez   

          City State ZIP/Post Code Reason Category        Reason Group  ...  \
0          NaN   NaN         00000         Request  Literature Request  ...   
1  George West   TX          78022        Question         Product use  ...   

        Marketing Segment Product Type Packaging Segment        Brand  \
0    Selective Herbicides  Concentrate            Bottle  BioAdvanced   
1  Wood Infesting Insects  Concentrate            Bottle  BioAdvanced   

                                           Case Text  \
0  From Address: SBM <global_internal_communicati...   
1  Caller has this product and she is spraying he...   

                                             Product   Disposition  \
0  All in One Lawn Weed & Crabgrass Killer 32 oz ...   SDS Request   
1     Carpenter Ant & Termite Killer Plus 40 oz Conc  Mixing ratio   

         Level 1                         Level 2            Level 3  
0  General Issue                Customer Request  Safety Data Sheet  
1  General Issue  Seeking Suggestion/Information       Mixing ratio  

[2 rows x 26 columns]

In [28]:
data.dropna(inplace=True)
data.isnull().sum()

Case ID              0
Issue Date           0
Time                 0
Contact Method       0
Full Name            0
City                 0
State                0
ZIP/Post Code        0
Reason Category      0
Reason Group         0
Reason Code          0
Product Group        0
Product Subgroup     0
Product Section      0
Product Code         0
Brand Manager        0
Marketing Segment    0
Product Type         0
Packaging Segment    0
Brand                0
Case Text            0
Product              0
Disposition          0
Level 1              0
Level 2              0
Level 3              0
dtype: int64

In [29]:
data['Reason Category'].value_counts()

Question                    15309
Complaint                    2062
Request                       450
Exposure                       77
Other Reason for Contact       37
Praise                         11
Suggestion                      9
Marketing Promotions            3
Name: Reason Category, dtype: int64

In [30]:
label = data['Reason Category']
data = data.drop(['Reason Category'],axis=1)

In [31]:
train_x, text_x , y_train, y_test = train_test_split(data,label)

In [32]:
train_x['Reason Category'] = y_train

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [35]:
data = train_x

In [40]:
data['Reason Category'].value_counts()

Question                    11443
Complaint                    1584
Request                       339
Exposure                       54
Other Reason for Contact       28
Praise                         10
Suggestion                      7
Marketing Promotions            3
Name: Reason Category, dtype: int64

In [41]:
question = data[data['Reason Category'] == 'Question']
complaint = data[data['Reason Category'] == 'Complaint']
request = data[data['Reason Category'] == 'Request']
exposure = data[data['Reason Category'] == 'Exposure']
other = data[data['Reason Category'] == 'Other Reason for Contact']
praise = data[data['Reason Category'] == 'Praise']
suggestion = data[data['Reason Category'] == 'Suggestion']
marketing = data[data['Reason Category'] == 'Marketing Promotions']


In [42]:
l = len(question)
print(l)

11443


In [43]:
question = resample(question,replace=True,n_samples = l,random_state = 123)
complaint = resample(complaint,replace=True,n_samples = l,random_state = 123)
request = resample(request,replace=True,n_samples = l,random_state = 123)
exposure = resample(exposure,replace=True,n_samples = l,random_state = 123)
other = resample(other,replace=True,n_samples = l,random_state = 123)
praise = resample(praise,replace=True,n_samples = l,random_state = 123)
suggestion = resample(suggestion,replace=True,n_samples = l,random_state = 123)
marketing = resample(marketing,replace=True,n_samples = l,random_state = 123)

In [44]:
marketing.shape

(11443, 26)

In [45]:
data_unsample =  pd.concat([question,complaint,request,exposure,other,praise,suggestion,marketing])

In [46]:
data_unsample.shape

(91544, 26)

In [62]:
print(data_unsample['Case Text'].head())

4628     Caller left a voicemail and has already been a...
14319    From Address: "Judy Corbett" <judycorbett@sbcg...
9792     Caller wants to know a product for Linden tree...
9597     Caller wanted to know if we had a product that...
879      Caller has this product and has termites in so...
Name: Case Text, dtype: object


In [54]:
def reg_exp(line):
    line = re.sub(r'[.\w-]+@[.\w-]+',"",str(line))#email
    line = re.sub(r'http://\S+',"",line)#url
    line = re.sub(r'[\[\])/(,.><]'," ",line)#punctuation
    line = re.sub(r'[\n+\s+]'," ",line)#newline and spaces
    return line

In [55]:
case_data = data_unsample["Case Text"]
lines = []
for line in data_unsample["Case Text"]:
    line = reg_exp(line)
#     line = re.sub('\w+:')
    lines.append(line)

In [70]:
lines_test = []
for line in text_x["Case Text"]:
    line = reg_exp(line)
    lines_test.append(line)

In [299]:
# train_x, text_x , y_train, y_test = train_test_split(lines,data_unsample['Reason Category'])

In [63]:
print(len(lines))

91544


In [64]:
tfid = TfidfVectorizer(max_df= 0.7,min_df=0.1,stop_words="english",lowercase= True, ngram_range=(1,3))

In [65]:
p = Pipeline([('tfid',tfid),('svm',LinearSVC())])

In [66]:
p.fit(lines,data_unsample['Reason Category'])

Pipeline(memory=None,
     steps=[('tfid', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.7, max_features=None, min_df=0.1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [67]:
test = ["where can i buy it. why is it so expensive?","I like this product. Its very good"]

In [68]:
prediction = p.predict(test)

In [69]:
print(prediction)

['Question' 'Exposure']


In [71]:
prediction = p.predict(lines_test)

In [72]:
accuracy_score(y_test,prediction)

0.5641425389755012

In [73]:
print(classification_report(y_test,prediction))

                          precision    recall  f1-score   support

               Complaint       0.45      0.69      0.55       478
                Exposure       0.04      0.52      0.07        23
    Marketing Promotions       0.00      0.00      0.00         0
Other Reason for Contact       0.00      0.11      0.01         9
                  Praise       0.00      0.00      0.00         1
                Question       0.97      0.55      0.70      3866
                 Request       0.18      0.59      0.27       111
              Suggestion       0.00      0.00      0.00         2

               micro avg       0.56      0.56      0.56      4490
               macro avg       0.21      0.31      0.20      4490
            weighted avg       0.89      0.56      0.67      4490



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [96]:
rc = RandomForestClassifier(max_depth=5)

In [97]:
p = Pipeline([('tfid',tfid),('rc',rc)])

In [98]:
p.fit(lines,data_unsample['Reason Category'])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('tfid', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.7, max_features=None, min_df=0.1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [99]:
prediction = p.predict(lines_test)

In [100]:
print(classification_report(y_test,prediction))

                          precision    recall  f1-score   support

               Complaint       0.46      0.48      0.47       478
                Exposure       0.02      0.65      0.05        23
    Marketing Promotions       0.00      0.00      0.00         0
Other Reason for Contact       0.01      0.11      0.01         9
                  Praise       0.00      0.00      0.00         1
                Question       0.97      0.57      0.72      3866
                 Request       0.11      0.61      0.19       111
              Suggestion       0.00      0.00      0.00         2

               micro avg       0.56      0.56      0.56      4490
               macro avg       0.20      0.30      0.18      4490
            weighted avg       0.89      0.56      0.67      4490



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [101]:
accuracy_score(y_test,prediction)

0.5583518930957684

In [85]:
test = ["where can i buy it."," it doesn't work.don't buy it"]

In [91]:
prediction = p.predict([reg_exp(test[1])])

In [92]:
print(prediction)

['Question']


In [102]:
#logistic regression

In [104]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('tfid',tfid),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])

In [105]:
logreg.fit(lines,data_unsample['Reason Category'])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Pipeline(memory=None,
     steps=[('tfid', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.7, max_features=None, min_df=0.1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True...penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False))])

In [106]:
predicted = logreg.predict(lines_test)

In [107]:
print(classification_report(y_test,predicted))

                          precision    recall  f1-score   support

               Complaint       0.46      0.68      0.55       478
                Exposure       0.04      0.48      0.07        23
Other Reason for Contact       0.00      0.11      0.01         9
                  Praise       0.00      0.00      0.00         1
                Question       0.97      0.58      0.73      3866
                 Request       0.16      0.56      0.25       111
              Suggestion       0.00      0.00      0.00         2

               micro avg       0.59      0.59      0.59      4490
               macro avg       0.23      0.34      0.23      4490
            weighted avg       0.89      0.59      0.69      4490



In [108]:
#naivebayes

In [110]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer

In [112]:
nb = Pipeline([('tfid', tfid),
               ('clf', MultinomialNB()),
              ])
nb.fit(lines,data_unsample['Reason Category'])

Pipeline(memory=None,
     steps=[('tfid', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.7, max_features=None, min_df=0.1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True...rue,
        vocabulary=None)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [117]:
predicted = nb.predict(lines_test)

In [118]:
print(classification_report(y_test,predicted))

                          precision    recall  f1-score   support

               Complaint       0.48      0.50      0.49       478
                Exposure       0.04      0.52      0.08        23
    Marketing Promotions       0.00      0.00      0.00         0
Other Reason for Contact       0.00      0.22      0.01         9
                  Praise       0.00      0.00      0.00         1
                Question       0.97      0.59      0.74      3866
                 Request       0.16      0.49      0.24       111
              Suggestion       0.00      0.00      0.00         2

               micro avg       0.58      0.58      0.58      4490
               macro avg       0.21      0.29      0.19      4490
            weighted avg       0.89      0.58      0.69      4490



In [128]:
test = ["where can i buy it?"," i hate the product."]
prediction = p.predict([reg_exp(test[1])])

In [129]:
print(prediction)

['Exposure']
